## EERIE ocean plotting

For the ocean you need a newer version of eccodes, and in general newer environment.

Easy way:

```shell

cd ~/.
mkdir -p .local/share/jupyter/kernels/iccp
cp /home/a/a270088/.local/share/jupyter/kernels/iccp/* .local/share/jupyter/kernels/iccp

```

Hard way:
You have to have mamba installed, for example following instructions on DKRZ: https://docs.dkrz.de/doc/levante/code-development/python.html#set-up-conda-for-individual-environments

```shell
mamba create -n iccp python=3.11
conda activate iccp

mamba install xarray dask cartopy matplotlib intake intake-xarray netcdf4 ipykernel scipy aiohttp intake-esm fastparquet

pip install eccodes
pip install gribscan

python -m ipykernel install --user --name iccp --display-name="iccp"

```

In [1]:
import dask

In [2]:
dask.__version__

'2025.11.0'

In [3]:
import eccodes

In [4]:
eccodes.codes_get_version_info(), eccodes.codes_get_library_path()

({'eccodes': '2.44.0', 'bindings': '2.44.0'},
 '/work/ab0995/a270088/mambaforge/envs/iccp/lib/python3.11/site-packages/eccodeslib/lib64/libeccodes.so')

In [5]:
import xarray as xr
import numpy as np
import intake

eerie_cat=intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")

model = 'ifs-fesom2-sr'
expid = 'hist-1950'  # 'highres-future-ssp245', 'hist-1950', 'eerie-control-1950', 'eerie-spinup-1950'
version = 'v20240304'    #
realm='ocean'  # 'ocean' , 'land', 'atmos'
gridspec = 'native' # 'native', 'gr025'

cat_data=eerie_cat['dkrz.disk.model-output'][model][expid][version][realm][gridspec]
print(list(cat_data))

# ds = cat_data['2D_daily_avg'].to_dask()
# ds

['2D_daily_avg', '3D_daily_avg', '3D_monthly_avg_elems']


In [6]:
ds = cat_data['2D_daily_avg'].to_dask()
ds

/work/ab0995/a270088/mambaforge/envs/iccp/lib/python3.11/site-packages/zarr/core/metadata/v2.py:190: ZarrUserWarning: Found an empty list of filters in the array metadata document. This is contrary to the Zarr V2 specification, and will cause an error in the future. Use None (or Null in a JSON document) instead of an empty list of filters.
  warnings.warn(msg, ZarrUserWarning, stacklevel=1)
/work/ab0995/a270088/mambaforge/envs/iccp/lib/python3.11/site-packages/zarr/core/metadata/v2.py:190: ZarrUserWarning: Found an empty list of filters in the array metadata document. This is contrary to the Zarr V2 specification, and will cause an error in the future. Use None (or Null in a JSON document) instead of an empty list of filters.
  warnings.warn(msg, ZarrUserWarning, stacklevel=1)


<xarray.Dataset> Size: 4TB
Dimensions:        (time: 7305, value: 7402886, node_index: 7402886)
Coordinates:
  * time           (time) datetime64[ns] 58kB 1950-01-01T12:00:00 ... 1969-12...
  * node_index     (node_index) int64 59MB 0 1 2 3 ... 7402883 7402884 7402885
Dimensions without coordinates: value
Data variables:
    avg_mlotst125  (time, value) float64 433GB dask.array<chunksize=(2, 7402886), meta=np.ndarray>
    avg_siconc     (time, value) float64 433GB dask.array<chunksize=(2, 7402886), meta=np.ndarray>
    avg_sisnthick  (time, value) float64 433GB dask.array<chunksize=(2, 7402886), meta=np.ndarray>
    avg_sithick    (time, value) float64 433GB dask.array<chunksize=(2, 7402886), meta=np.ndarray>
    avg_siue       (time, value) float64 433GB dask.array<chunksize=(2, 7402886), meta=np.ndarray>
    avg_sivn       (time, value) float64 433GB dask.array<chunksize=(2, 7402886), meta=np.ndarray>
    avg_sos        (time, value) float64 433GB dask.array<chunksize=(2, 7402886), meta=np.ndarray>
    avg_tos        (time, value) float64 433GB dask.array<chunksize=(2, 7402886), meta=np.ndarray>
    avg_zos        (time, value) float64 433GB dask.array<chunksize=(2, 7402886), meta=np.ndarray>
Attributes: (12/37)
    edition:               2
    centre:                ecmf
    centreDescription:     European Centre for Medium-Range Weather Forecasts
    subCentre:             1003
    uuidOfHGrid:           7a33ecae-be22-e7d5-ca4a-bbd819a43240
    history:               🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan...
    ...                    ...
    parent_activity_id:    HighResMIP
    sub_experiment_id:     none
    experiment:            coupled historical 1950-2014
    source:                IFS-FESOM2-SR (2023): \naerosol: none, prescribed ...
    institution:           Alfred Wegener Institute, Helmholtz Centre for Pol...
    sub_experiment:        none

In [7]:
from dask.distributed import Client
import dask
dask.config.config.get('distributed').get('dashboard').update({'link':'{JUPYTERHUB_SERVICE_PREFIX}/proxy/{port}/status'})
dask.config.set({'temporary_directory': '/scratch/a/a270088/dask/'})
# client = Client(n_workers=5, threads_per_worker=1, memory_limit='50GB')
client = Client(n_workers=5, threads_per_worker=1, memory_limit='20GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Workers: 5
Total threads: 5,Total memory: 93.13 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34027,Workers: 0
Dashboard: /user/a270088/levante-spawner-preset//proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:35087,Total threads: 1
Dashboard: /user/a270088/levante-spawner-preset//proxy/43139/status,Memory: 18.63 GiB
Nanny: tcp://127.0.0.1:44407,


In [ ]:
%%time
temp_std = ds['avg_tos'].sel(time=slice('1950','1951')).std(dim='time').compute()
